In [2]:
import urllib
import os.path
from textgenrnn import textgenrnn
from bs4 import BeautifulSoup
import pickle
import sys

sys.setrecursionlimit(20000)  # for pickling scraper pages

Using TensorFlow backend.


In [3]:
pages_file = 'pages.pickle'

def get_poem_pages():
    if os.path.isfile(pages_file):
        with open(pages_file, 'rb') as f: 
            return pickle.load(f)
    pages = []
    for i in range(1, 7):
        index = BeautifulSoup(urllib.request.urlopen('https://www.poemhunter.com/shel-silverstein-2/poems/page-' + str(i)), 'html.parser')
        urls = ['https://www.poemhunter.com' + td.contents[1].get('href') for td in index.find_all('td', attrs={'class': 'title'})]
        for url in urls:
            poem_soup = BeautifulSoup(urllib.request.urlopen(url), 'html.parser')
            pages.append(poem_soup)
    return pages

In [4]:
pages = get_poem_pages()

In [5]:
with open(pages_file, 'wb') as f:  
    pickle.dump(pages, f)

In [25]:
def parse_poems(pages):
    poems = []
    for page in pages:
        poem_lines = page.find('div', attrs={'class': 'KonaBody'}).findChildren('p')[0].contents
        poem_dirty = u''.join([str(line).strip() for line in poem_lines]).strip()
        poem_clean = poem_dirty.replace('<br/><br/>', ' || ').replace('<br/>', ' / ')
        poems.append(poem_clean)
    return poems

In [30]:
poems = parse_poems(pages)

In [27]:
textgen = textgenrnn()
textgen.train_on_texts(poems, num_epochs=2,  gen_epochs=2)

Training on 206,105 character sequences.
Epoch 1/2
1610/1610 [==============================] - 412s 256ms/step - loss: 1.5843
Epoch 2/2
1610/1610 [==============================] - 428s 266ms/step - loss: 1.3817
####################
Temperature: 0.2
####################
I may hear the started the started the country / And I think I'll be a lot of the stone / And I'm sticked to the tear the store / And I have to the time I got an ape of the season / And I'm gonna be a lot of the street / And the started lookin' and he says he's got a little time / And I'm gonna lo

I would start to say I'll be might look at the stop / And I say I was a little bit more the way / And I think I think I'm gonna find / I got an and the bottom and the country / And the country was a brand to me / And I'll be a poor place on the way to the store / And I may be my house and I can't 

Well I mean the little time to the world / And I think I'm still and I think I'm a lot of the started / And I may be a lot of th

In [ ]:
generated = len(os.listdir('lambda/poems'))
rnn = textgenrnn('shel_model_weights.hdf5')
while True:
    with open('lambda/poems/' + str(generated) + '.txt', 'w') as f:
        poem = rnn.generate(n=1, temperature=0.5, return_as_list=True)[0]
        f.write(poem)
        generated += 1